In [1]:
import argparse
import errno
import glob
import json
import os
import re
import shutil
from types import SimpleNamespace
import cv2
import torch
import warnings
from lightning_objects import LightningModel
warnings.filterwarnings('ignore')
from config import Configuration
import pandas as pd
from utils import stratify_split, make_holdout_df, set_seeds
from train_manager import TrainManager

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def main(experiment_name: str, debug, resume=False,
         finetune=False, freeze_bn=True, freeze_feature_extractor=False):

    experiment_dir = os.path.abspath(f'trained-models/{experiment_name}')
    print('Experiment directory', experiment_dir)

    try:
        # -------- SETUP --------
        checkpoint_params = None
        finetune_model_fnames = None
        folds_df, holdout_df = None, None

        if not resume and not finetune: # totally new experiment
            make_experiment_directory(experiment_dir)
            config = Configuration()
            config.debug = debug
            set_seeds(config.seed)

            # -------- LOAD DATA FROM TRAIN FILE --------
            data_df = pd.read_csv(config.data_dir + '/train.csv', engine='python')
            data_df, holdout_df = make_holdout_df(data_df, seed=config.seed)
            folds_df = stratify_split(data_df, config.fold_num, config.seed, config.target_col)

            # -------- SAVE FILES (experiment state: things like resuming, fine tuning, and inference on holdout) --------
            folds_df.to_csv(experiment_dir + '/folds.csv', index=False)
            holdout_df.to_csv(experiment_dir + '/holdout.csv', index=False)
            with open(experiment_dir + '/experiment_config.json', 'w') as f:
                json.dump(config.__dict__, f)
        elif resume or finetune:
            # LOAD DATA FROM SAVED FILES
            with open(experiment_dir + '/experiment_config.json', 'r') as f:
                config = json.load(f, object_hook=lambda d: SimpleNamespace(**d))
                set_seeds(config.seed)
                config.debug = debug

            folds_df = pd.read_csv(experiment_dir + '/folds.csv', engine='python')
            holdout_df = pd.read_csv(experiment_dir + '/holdout.csv', engine='python')

            if finetune and not resume:
                print('finetuning...')
                # verify there are checkpoints to fine tune
                finetune_model_fnames = glob.glob(experiment_dir + '/*fold*.ckpt')
                assert len(finetune_model_fnames) > 0
                finetune_model_fnames.sort()

                # make new directory for tuning experiment with files from training run 1
                make_experiment_directory(experiment_dir + '_tune')
                for f in os.listdir(experiment_dir):
                    print(f"copying {f} to {experiment_dir + '_tune'}")
                    shutil.copy2(experiment_dir + '/' + f, experiment_dir + '_tune')
                experiment_dir += '_tune'
                experiment_name += '_tune'
            else:
                print('resuming from last checkpoint...')
                checkpoint_params = get_checkpoint_params(experiment_dir, resume)

        assert holdout_df is not None, 'holdout_df is None'
        assert folds_df is not None, 'folds_df is None'

        # cv2 multithreading seems to go into deadlock with PyTorch data loaders
        if config.num_workers > 0:
            cv2.setNumThreads(0)

        trainer = TrainManager(experiment_name=experiment_name, experiment_dir=experiment_dir,
                               folds_df=folds_df, holdout_df=holdout_df,
                               checkpoint_params=checkpoint_params, config=config,
                               finetune=finetune, freeze_bn=freeze_bn,
                               freeze_feature_extractor=freeze_feature_extractor,
                               finetune_model_fnames=finetune_model_fnames)
        trainer.run()
    finally:
        torch.cuda.empty_cache()

def make_experiment_directory(name):
    try:
        os.makedirs(name)
    except FileExistsError as e:
        print('Experiment already exists. Be sure to resume training appropriately or start a new experiment.')
        if e.errno == errno.EEXIST: raise


def get_checkpoint_params(basename, resume):
    checkpoint_params = None
    if resume:
        checkpoint_params = {}
        model_filenames = glob.glob(basename + '/*fold*.ckpt')
        model_filenames.sort()
        trained_folds = [re.findall(r'fold\d+', f)[0][len('fold'):] for f in model_filenames]
        most_recent_fold = int(max(trained_folds)) if len(trained_folds) > 0 else 0

        checkpoint_params['restart_from'] = most_recent_fold
        checkpoint_params['checkpoint_file_path'] = model_filenames[-1]

    return checkpoint_params

In [4]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Kaggle Cassava Disease Training')

    try:
        debug = False
        print('Running in debug mode:', debug)
        main(experiment_name='tf_efficientnet_b3a_ns_sgd_1cycle_fcnodes512_256_smoothing=0.05_weighted_headonly', debug=debug,
             resume=False, finetune=False, freeze_bn=True, freeze_feature_extractor=True)
    except KeyboardInterrupt:
        pass

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Running in debug mode: False
Experiment directory /opt/favordata/AI/Felix/kaggle-cassava/trained-models/tf_efficientnet_b3a_ns_sgd_1cycle_fcnodes512_256_smoothing=0.05_weighted_headonly
folds_df len 18187, holdout_df len 3210
Training fold 0
Class sample counts [ 758 1470 1623 8933 1765]
After class sample counts [2274 2940 3732 8933 4765]
EfficientNet(
  (conv_stem): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
        (bn1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
          (a


  | Name           | Type               | Params
------------------------------------------------------
0 | valid_accuracy | Accuracy           | 0     
1 | test_accuracy  | Accuracy           | 0     
2 | criterion      | LabelSmoothingLoss | 0     
3 | model          | EfficientNet       | 11.6 M
------------------------------------------------------
919 K     Trainable params
10.7 M    Non-trainable params
11.6 M    Total params


Epoch 0:  89%|████████▉ | 455/512 [01:59<00:14,  3.82it/s, loss=43.9, v_num=3, val_loss=46, val_acc=0, train_loss=44]  
Validating: 0it [00:00, ?it/s]
Validating: 100%|██████████| 57/57 [00:30<00:00,  1.52it/s]

Epoch 0, global step 113: val_loss reached 43.80122 (best 43.80122), saving model to "/opt/favordata/AI/Felix/kaggle-cassava/trained-models/tf_efficientnet_b3a_ns_sgd_1cycle_fcnodes512_256_smoothing=0.05_weighted_headonly/efficientnet_b3a_bitempered_smooth=0.05_val_loss=43.801_val_acc=0.687_fold0.ckpt" as top 1


...

Epoch 26:  89%|████████▉ | 456/512 [01:59<00:14,  3.83it/s, loss=43.5, v_num=3, val_loss=43.3, val_acc=0.794, train_loss=43.3]
Validating: 0it [00:00, ?it/s]
Validating: 100%|██████████| 57/57 [00:28<00:00,  2.48it/s]

Epoch 26, step 3077: val_loss was not in top 1


Epoch 27:  89%|████████▉ | 456/512 [01:59<00:14,  3.83it/s, loss=43.4, v_num=3, val_loss=43.3, val_acc=0.787, train_loss=43.3]
Validating: 0it [00:00, ?it/s]
Validating: 100%|██████████| 57/57 [00:28<00:00,  2.48it/s]

Epoch 27, step 3191: val_loss was not in top 1


Epoch 27: 100%|██████████| 512/512 [02:28<00:00,  3.45it/s, loss=43.4, v_num=3, val_loss=43.6, val_acc=0.786, train_loss=43.5]
